In [19]:
from typing import TypedDict, Annotated

from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, END, add_messages
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from langgraph.prebuilt import ToolNode

load_dotenv()
class ChildState(TypedDict):
    messages: Annotated[list, add_messages]

tavily_search = TavilySearch(max_results=3)
tools = [tavily_search]

llm = ChatOpenAI(model="doubao-seed-1-6-lite-251015")
llm_with_tool = llm.bind_tools(tools)
def agent(state: ChildState):
    return {
        "messages": [llm_with_tool.invoke(state["messages"])]
    }

def tools_router(state: ChildState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
        return "tool_node"
    return END
tool_node = ToolNode(tools=tools)

subgraph = StateGraph(ChildState)
subgraph.add_node("agent", agent)
subgraph.add_node("tool_node", tool_node)
subgraph.set_entry_point("agent")

subgraph.add_conditional_edges("agent", tools_router)
subgraph.add_edge("tool_node", "agent")

search_app = subgraph.compile()

## stream_mode="values"-> 不断追加新的消息,包含之前地旧值

#### 每个event包含{event：事件类型，name：时间名， data: 涉及到地数据，metadata：涉及到langgraph_node的元数据}

In [21]:
from langchain_core.messages import AIMessageChunk
input_human = {"messages": [
    HumanMessage(content="帮我查找一下关于人工智能的最新发展")
]}
# events = search_app.stream(input=input_human, stream_mode="values")
# for event in events:
#     print(event)

for message_chunk, metadata in search_app.stream(input=input_human, stream_mode="messages"):
    if message_chunk.content and isinstance(message_chunk, AIMessageChunk):
        print(message_chunk.content, end="", flush=True)

以下是2024年人工智能领域的最新发展核心动态：


### **一、技术突破**
1. **多模态与生成能力跃升**  
   - OpenAI发布视频生成模型**Sora**（内测），可生成1分钟高清视频；  
   - 谷歌**Gemini 1.5 Pro**实现100万token上下文输入，强化多模态处理；  
   - 中国科学院指出「**小数据+优质数据**」成为AI研究新焦点（替代单纯追求数据规模）。

2. **脑机接口与具身智能**  
   - Neuralink首次展示患者通过**意念操控游戏**，验证医疗康复潜力；  
   - 特斯拉发布**Cybercab、Robovan、Tesla Bot**，推动AI在交通/家庭场景落地。

3. **算力与量子计算**  
   - 马斯克xAI建成全球最大超级计算机「**巨像（Colossus）**」；  
   - 谷歌量子芯片「**Willow**」实现量子优越性（传统计算机需数十亿年的任务仅需几分钟）。


### **二、应用落地（从「百模大战」转向「AI+行业」）**
- **工业领域**：百度AI赋能污水厂精准投药（节约15%成本）、中小企业分拣/质检效率提升；  
- **能源/原材料**：宝钢用AI优化高炉冶炼（降能耗、提质量）、瑞典Recycleye实现废弃物智能分类；  
- **金融/医疗/教育**：行业大模型已实现初步应用并产生经济效益（信通院《2024AI发展报告》）。


### **三、产业生态**
- **投融资**：2024上半年全球AI融资316亿美元（同比+84%），占全行业比例从2022年4.5%升至12.1%；  
- **学术认可**：2024诺贝尔物理学/化学奖均与AI研究相关；  
- **政策推动**：中国国务院部署AI赋能新型工业化，全球多国将AI列为国家战略核心。


### **四、关键趋势**
AI正从「通用技术」向「底层学科」演进，有望重构社会物理/虚拟空间（中国工程院院士郑南宁）。


数据来源：新华网、中国信通院《2024AI发展报告》、博客园2024AI大事件汇总、中国科学院前沿趋势报告。

## stream_mode="updates"-> 每次只返回最新的消息

In [31]:
input_human = {"messages": [
    HumanMessage(content="帮我查找一下关于人工智能的最新发展")
]}
events = search_app.stream(input=input_human, stream_mode="updates")
for event in events:
    print(event)
    print("\n\n")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 182, 'prompt_tokens': 2049, 'total_tokens': 2231, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 152, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-lite-251015', 'system_fingerprint': None, 'id': '021762159863010ad114934056b0c06cfdb66f2980f6adc480830', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--ab6849fa-4ce5-4f38-b926-c508b6c2e688-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '人工智能最新发展 2024', 'search_depth': 'advanced'}, 'id': 'call_cm5h0ywncydbmf78quaiymp2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2049, 'output_tokens': 182, 'total_tokens': 2231, 'input_token_details': {'cache_read': 0}, 'output_t

## Shared Schema

In [28]:
class ParentState(TypedDict):
    messages: Annotated[list, add_messages]

parent_graph = StateGraph(ParentState)

parent_graph.add_node("search_subgraph", search_app)
parent_graph.set_entry_point("search_subgraph")
parent_graph.add_edge("search_subgraph", END)

parent_app = parent_graph.compile()

input_human = {"messages": [
    HumanMessage(content="帮我查找一下关于人工智能的最新发展")
]}
events = parent_app.stream(input=input_human, stream_mode="values")
for event in events:
    print(event)

{'messages': [HumanMessage(content='帮我查找一下关于人工智能的最新发展', additional_kwargs={}, response_metadata={}, id='26d77def-0ad7-4604-8a81-d735fc9029a5')]}



{'messages': [HumanMessage(content='帮我查找一下关于人工智能的最新发展', additional_kwargs={}, response_metadata={}, id='26d77def-0ad7-4604-8a81-d735fc9029a5'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 2049, 'total_tokens': 2196, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 122, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-lite-251015', 'system_fingerprint': None, 'id': '0217621593563162856367a65b621c15f7d0cb2a0f35cc4dd647a', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--dfce8acc-2df8-4f11-96d1-60e2c46558b1-0', tool_calls=[{'name': 'tavily_

## Different Schema

In [25]:
class QueryState(TypedDict):
    query: str
    response: str

def SearchAgent(state: QueryState):
    query = state["query"]
    subgraph_input = {"messages": [HumanMessage(content=query)]}
    subgraph_output = search_app.invoke(subgraph_input)
    response = subgraph_output["messages"][-1].content
    return {"response": response}

parent_graph = StateGraph(QueryState)
parent_graph.add_node("search_agent", SearchAgent)
parent_graph.set_entry_point("search_agent")
parent_graph.add_edge("search_agent", END)
parent_app = parent_graph.compile()
parent_app.invoke({
    "query": "帮我查找一下关于人工智能的最新发展"
})["response"]

'以下是2025年10-11月人工智能领域的最新发展动态，综合国内政策、全球产业布局及技术趋势整理：\n\n\n### **一、国内AI发展核心进展**\n1. **产业体系与用户规模爆发**  \n   - 形成**基础层（芯片/算力）→框架层→模型层→应用层**完整产业链；  \n   - 生成式AI用户规模达**5.15亿人**（2025年6月），半年翻番，**90%以上用户首选国产大模型**；  \n   - AI专利申请量全球第一（157.6万件，占全球38.58%）。\n\n2. **政策与应用导向升级**  \n   - 国务院印发《**人工智能+行动意见**》，明确：  \n     → 2027年：AI与6大重点领域深度融合，智能终端普及率超70%；  \n     → 2030年：智能经济成为核心增长极，普及率超90%；  \n   - 具身智能（机器人+AI）成投融资热点，落地教育、医疗、线下服务场景。\n\n\n### **二、全球10月关键动态**\n1. **基础设施竞赛白热化**  \n   - 美国科技巨头AI基础设施支出超**4000亿美元**；  \n   - OpenAI+博通联合开发**10吉瓦算力定制AI加速器**；  \n   - 贝莱德财团以**400亿美元收购全球最大数据中心运营商**；  \n   - 谷歌重启核电站为AI供电，Meta计划建1吉瓦数据中心。\n\n2. **政策与应用创新**  \n   - 欧盟发布「应用AI」「科学AI」两项战略；  \n   - 中国出台《政务领域AI大模型部署指引》；  \n   - 意大利用AI守护文物安防，斯坦福举办**全球首个全AI撰写/评审的学术会议**。\n\n3. **产品突破**  \n   - OpenAI推出**ChatGPT Atlas浏览器**，挑战谷歌Chrome；  \n   - 微软与OpenAI签署长期协议，支持OpenAI转型公益公司（PBC）。\n\n\n### **三、麦肯锡解析核心技术趋势**\n1. **代理式AI**：从文本模型转向**自主行动的智能代理**（如专业研究代理）；  \n2. **专用半导体**：英伟达等主导AI芯片创新，支撑算力需求；  \n3. **高级连接**：5G/6G与AI融合，加速边缘计算落地。\n\n\n#